<a href="https://colab.research.google.com/github/Deep-Learning-Qatar/EEG-Vision/blob/master/Pipeline_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Pipeline**: This is a general set-up to evaluate our entire EEG-Vision predictive system using the thre individual models that are imported to maximize modularity.

In [1]:
# mount gdrive
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


# Imports

In [1]:
# import libraries
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data

import matplotlib.pyplot as plt
import random

In [2]:
# install and import wandb
!pip install wandb --upgrade
import wandb
wandb.login()

Requirement already up-to-date: wandb in /usr/local/lib/python3.7/dist-packages (0.10.29)


wandb: Currently logged in as: sbaumann (use `wandb login --relogin` to force relogin)


True

In [3]:
%cd /gdrive/MyDrive/11-785 Deep Learning/Project

# path to data
EEG_dataset_path = '/gdrive/MyDrive/11-785 Deep Learning/Project/EEG_datasets.pth'
EEG_out_dataset_path = '/gdrive/MyDrive/11-785 Deep Learning/Project/EEG_out_test_datasets_'
pth = '.pth'
Image_dataset_path = '/gdrive/MyDrive/11-785 Deep Learning/Project/data_by_image.pth'

from eeg_net import make_EEG_Net, train_and_val_EEG_Net, test_EEG_Net
from image_net import make_Image_Net, train_and_val_Image_Net, test_Image_Net, make_EEG_to_Image_data_loaders, get_class_labels
from eeg_image_map import make_EEG_Image_Map, train_and_val_EEG_Image_Map, test_EEG_Image_Map, calc_map_metrics

/gdrive/MyDrive/11-785 Deep Learning/Project


# Pipeline EEG-Net

In [5]:
def model_pipeline_EEG_Net(hyperparameters):
    %cd /gdrive/MyDrive/11-785 Deep Learning/Project

    with wandb.init(project='EEG-Net', entity='eegvision', config=hyperparameters):
      config = wandb.config

      if config.num_classes == 40: EEG_path = EEG_dataset_path
      else: EEG_path = EEG_out_dataset_path + str(40-config.num_classes) + pth

      model, EEG_data_loaders, criterion, optimizer, scheduler = make_EEG_Net(config, EEG_path)

      %cd EEG-Net
      model, val_acc_hist = train_and_val_EEG_Net(wandb, config, model, EEG_data_loaders, criterion, optimizer, scheduler, num_epochs=config.num_epochs)

    return model, EEG_data_loaders, val_acc_hist

In [6]:
EEG_hyperparams = dict(
    model_nr = 13,

    # Models to choose from [fares]
    model = "fares",

    # Number of classes in the dataset [40, 37, 35, 30, 20]
    num_classes = 35,

    # Number of epochs to train for
    num_epochs = 80,

    num_hidden_nodes = 256,

    num_features = 128,

    num_layers = 2,

    lr = 0.001,
    
    weight_decay = 5e-6,

    dropout = 0.4,

    batch_size = 64,
)

EEG_Net, EEG_data_loaders, eeg_acc_hist = model_pipeline_EEG_Net(EEG_hyperparams)

/gdrive/MyDrive/11-785 Deep Learning/Project


/gdrive/MyDrive/11-785 Deep Learning/Project/EEG-Net
Epoch 2/80
----------
[train] Loss: 3.1475 Acc: 0.0828
[val] Loss: 3.3086 Acc: 0.0805
Epoch 10/80
----------
[train] Loss: 2.3615 Acc: 0.2131
[val] Loss: 2.3898 Acc: 0.1906
Epoch 18/80
----------
[train] Loss: 1.7737 Acc: 0.3729
[val] Loss: 1.8750 Acc: 0.3065
Epoch 26/80
----------
[train] Loss: 1.2290 Acc: 0.5301
[val] Loss: 1.5304 Acc: 0.3937
Epoch    32: reducing learning rate of group 0 to 5.0000e-04.
Epoch 34/80
----------
[train] Loss: 0.7560 Acc: 0.7123
[val] Loss: 1.9313 Acc: 0.4339
Epoch    39: reducing learning rate of group 0 to 2.5000e-04.
Epoch 42/80
----------
[train] Loss: 0.4473 Acc: 0.8338
[val] Loss: 1.5996 Acc: 0.4943
Epoch    45: reducing learning rate of group 0 to 1.2500e-04.
Epoch 50/80
----------
[train] Loss: 0.2726 Acc: 0.9037
[val] Loss: 1.7945 Acc: 0.5220
Epoch    51: reducing learning rate of group 0 to 6.2500e-05.
Epoch    57: reducing learning rate of group 0 to 3.1250e-05.
Epoch 58/80
----------
[train

epoch,79
train_loss,0.16386
train_acc,0.94799
_runtime,2994
_timestamp,1620347301
_step,159
val_loss,1.87555
val_acc,0.52682


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train_loss,█▇▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▂▂▂▂▂▃▃▃▄▄▄▄▅▅▅▆▆▆▇▇▇▇█████████████████
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
val_loss,█▆▄▄▄▅▃▂▄▂▂▁▂▁▁▂▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
val_acc,▁▂▂▃▃▃▄▅▄▅▅▆▅▆▆▆▇▇▇▇▇▇▇█████████████████


In [ ]:
# EEG_Net.load_state_dict(torch.load('/gdrive/MyDrive/11-785 Deep Learning/Project/EEG-Net/EEGNet12epoch66.pth'))

# Pipeline Image-Net

In [7]:
def model_pipeline_Image_Net(hyperparameters):
    %cd /gdrive/MyDrive/11-785 Deep Learning/Project

    with wandb.init(project='Image-Net', entity='eegvision', config=hyperparameters):
      config = wandb.config
      
      if EEG_hyperparams['num_classes'] == 40: EEG_path = EEG_dataset_path
      else: EEG_path = EEG_out_dataset_path + str(40-EEG_hyperparams['num_classes']) + pth

      model, Image_data_loaders, criterion, optimizer = make_Image_Net(config, EEG_path, Image_dataset_path)

      %cd Image-Net
      model, val_acc_hist = train_and_val_Image_Net(wandb, config, model, Image_data_loaders, criterion, optimizer,
                                                    is_inception=(config.model_name=="inception"))

    return model, Image_data_loaders, val_acc_hist

In [8]:
Image_hyperparams = dict(
    model_nr = 2,

    # Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
    model_name = "inception",

    # Number of classes in the dataset
    num_classes = 40,

    num_features = 512,

    # Batch size for training (change depending on how much memory you have)
    batch_size = 64,

    # Number of epochs to train for
    num_epochs = 0,

    # Flag for feature extracting. When False, we finetune the whole model,
    #   when True we only update the reshaped layer params
    feature_extract = True,

    lr = 0.001,

    relu = True,
)

Image_Net, Image_data_loaders, image_acc_hist = model_pipeline_Image_Net(Image_hyperparams)

/gdrive/MyDrive/11-785 Deep Learning/Project


ImageNet(
  (pretrained): Inception3(
    (Conv2d_1a_3x3): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2a_3x3): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2b_3x3): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (Conv2d_3b_1x1): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_4a_3x3): Bas

In [9]:
Image_Net.load_state_dict(torch.load('/gdrive/MyDrive/11-785 Deep Learning/Project/Image-Net/ImageNet2epoch14.pth'))

<All keys matched successfully>

# Create feature dataset

In [10]:
nr_eeg_features, nr_image_features = 0, 0
new_Image_data_loaders = None

In [11]:
def make_feature_datasets(EEG_data_loaders, Image_data_loaders):
    global nr_eeg_features, nr_image_features, new_Image_data_loaders

    input_size = 299 if Image_hyperparams['model_name'] == 'inception' else 224
    new_Image_data_loaders = make_EEG_to_Image_data_loaders(EEG_data_loaders, Image_dataset_path, input_size)
    
    for phase in ['train_unshuffle', 'val', 'test']:
        # Pass in train, valid, test data through EEG-Net to get EEG features
        _, eeg_features, _ = test_EEG_Net(EEG_Net, EEG_data_loaders[phase])
        eeg_features = [item for sublist in eeg_features for item in sublist]
        nr_eeg_features = len(eeg_features[0])

        # Pass in train, valid, test data through Image-Net to get Image features
        _, image_features, _ = test_Image_Net(Image_Net, new_Image_data_loaders[phase])
        image_features = [item for sublist in image_features for item in sublist]
        nr_image_features = len(image_features[0])

        # Get class labels for each feature embedding pair
        class_labels = get_class_labels(new_Image_data_loaders[phase])
        class_labels = [item for sublist in class_labels for item in sublist]

        data_path = phase + '_data_' + str(EEG_hyperparams['model_nr']) + '.npy'
        np.save(data_path, np.array(eeg_features))

        labels_path = phase + '_labels_' + str(EEG_hyperparams['model_nr']) + '.npy'
        np.save(labels_path, np.array(image_features))

        class_labels_path = phase + '_class_labels_' + str(EEG_hyperparams['model_nr']) + '.npy'
        np.save(class_labels_path, np.array(class_labels))

        print(data_path, labels_path, class_labels_path)


In [12]:
%cd /gdrive/MyDrive/11-785 Deep Learning/Project
make_feature_datasets(EEG_data_loaders, Image_data_loaders)

print(nr_eeg_features, nr_image_features)

/gdrive/MyDrive/11-785 Deep Learning/Project
train_unshuffle_data_13.npy train_unshuffle_labels_13.npy train_unshuffle_class_labels_13.npy
val_data_13.npy val_labels_13.npy val_class_labels_13.npy
test_data_13.npy test_labels_13.npy test_class_labels_13.npy
128 512


# Pipeline EEG-Image-Map

In [13]:
def model_pipeline_EEG_Image_Map(hyperparameters):
    %cd /gdrive/MyDrive/11-785 Deep Learning/Project

    with wandb.init(project='EEG-Image-Map', entity='eegvision', config=hyperparameters):
      config = wandb.config
      model, feature_data_loaders, criterion, optimizer, scheduler = make_EEG_Image_Map(config)

      %cd EEG-Image-Map
      model, val_acc_hist = train_and_val_EEG_Image_Map(wandb, config, model, feature_data_loaders, criterion, optimizer, scheduler, num_epochs=config.num_epochs)

    return model, feature_data_loaders, val_acc_hist, criterion

In [16]:
feature_hyperparams = dict(
    model_nr = 21,

    # Models to choose from [mlp, bilstm]
    model = "mlp",

    train_data_path   = 'train_unshuffle_data_13.npy',
    train_labels_path = 'train_unshuffle_labels_13.npy',
    train_class_labels_path = 'train_unshuffle_class_labels_13.npy',
    val_data_path     = 'val_data_13.npy',
    val_labels_path   = 'val_labels_13.npy',
    val_class_labels_path = 'val_class_labels_13.npy',
    test_data_path    = 'test_data_13.npy',
    test_labels_path  = 'test_labels_13.npy',
    test_class_labels_path = 'test_class_labels_13.npy',
    bidirectional    = True,
    lr               = 0.00001,
    wd               = 5e-6,
    dropout_prob     = 0.1,
    num_layers       = 1,
    hidden_size      = 50,
    num_classes      = nr_image_features, #? (same as extractec image features)
    embiddings_num   = nr_eeg_features,  #  (same as number of features extracted from EEG_Net)
    batch_size       = 64,
    patience         = 1,
    scheduler_factor = 0.5,
    num_epochs       = 50,
    mlp_layers       = [nr_eeg_features, 256, 256, 512, nr_image_features],
    criterion        = 'cosine',
    scheduler        = 'plateau',
    step_size        = 50,
)

EEG_Image_Map, feature_data_loaders, val_acc_hist, feature_criterion = model_pipeline_EEG_Image_Map(feature_hyperparams)

/gdrive/MyDrive/11-785 Deep Learning/Project


/gdrive/MyDrive/11-785 Deep Learning/Project/EEG-Image-Map
Epoch 2/50
----------
[train] Loss: 0.4102
[val] Loss: 0.2973
Epoch 7/50
----------
[train] Loss: 0.1511
[val] Loss: 0.1260
Epoch 12/50
----------
[train] Loss: 0.1145
[val] Loss: 0.1011
Epoch 17/50
----------
[train] Loss: 0.1001
[val] Loss: 0.0919
Epoch 22/50
----------
[train] Loss: 0.0923
[val] Loss: 0.0873
Epoch 27/50
----------
[train] Loss: 0.0868
[val] Loss: 0.0843
Epoch 32/50
----------
[train] Loss: 0.0822
[val] Loss: 0.0822
Epoch 37/50
----------
[train] Loss: 0.0782
[val] Loss: 0.0804
Epoch 42/50
----------
[train] Loss: 0.0743
[val] Loss: 0.0794
Epoch 47/50
----------
[train] Loss: 0.0708
[val] Loss: 0.0786
Training complete in 1m 55s
Best val Loss: 0.078297


epoch,49
train_loss,0.06876
_runtime,118
_timestamp,1620347968
_step,99
val_loss,0.07844


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▅▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,█▅▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [15]:
EEG_Image_Map.load_state_dict(torch.load('/gdrive/MyDrive/11-785 Deep Learning/Project/EEG-Image-Map/EEGImageMap20epoch150.pth'))

<All keys matched successfully>

# Evaluate System

In [19]:
def evaluate_system(phase='test', verbose=True):
    # # Evaluate EEG-Net
    _, _, eeg_acc = test_EEG_Net(EEG_Net, EEG_data_loaders[phase])

    # # Evaluate Image-Net
    _, _, image_acc = test_Image_Net(Image_Net, Image_data_loaders[phase])

    # Evaluate EEG-Image-Map

    feature_out = test_EEG_Image_Map(EEG_Image_Map, feature_data_loaders[phase])
    feature_out = [item for sublist in feature_out for item in sublist]

    feature_id_acc, feature_class_acc, similarities = calc_map_metrics(feature_out, feature_data_loaders[phase])
    avg_similarity = sum(similarities)/len(similarities)

    if verbose:
        print('EEG-Net ' + phase + '  accuracy:', round(eeg_acc*100, 4), '%')
        print('Image-Net ' + phase + '  accuracy:', round(image_acc*100, 4), '%')
        print('EEG-Image-Map avg ' + phase + ' cosine similarity:', round(avg_similarity, 4))
        print('EEG-Image-Map ' + phase + ' classification accuracy:', round(feature_class_acc*100, 6), '%')
        print('EEG-Image-Map ' + phase + ' identification accuracy:', round(feature_id_acc*100, 6), '%')

    return eeg_acc, image_acc, avg_similarity, feature_class_acc, feature_id_acc

def evaluate_out_system(phase='test', verbose=True):
    # Evaluate EEG-Net
    phase_eeg = 'val' if phase == 'test' else phase
    _, _, eeg_acc = test_EEG_Net(EEG_Net, EEG_data_loaders[phase_eeg])

    # Evaluate Image-Net
    _, _, image_acc = test_Image_Net(Image_Net, Image_data_loaders[phase])

    # Evaluate EEG-Image-Map
    feature_out = test_EEG_Image_Map(EEG_Image_Map, feature_data_loaders[phase])
    feature_out = [item for sublist in feature_out for item in sublist]

    feature_id_acc, feature_class_acc, similarities = calc_map_metrics(feature_out, feature_data_loaders[phase])
    avg_similarity = sum(similarities)/len(similarities)

    if verbose:
        print('EEG-Net ' + phase_eeg + ' accuracy:', round(eeg_acc*100, 4), '%')
        print('Image-Net ' + phase + ' accuracy:', round(image_acc*100, 4), '%')
        print('EEG-Image-Map avg ' + phase + ' cosine similarity:', round(avg_similarity, 4))
        print('EEG-Image-Map ' + phase + ' classification accuracy:', round(feature_class_acc*100, 6), '%')
        print('EEG-Image-Map ' + phase + ' identification accuracy:', round(feature_id_acc*100, 6), '%')

    return eeg_acc, image_acc, avg_similarity, feature_class_acc, feature_id_acc


In [21]:
if EEG_hyperparams['num_classes'] == 40:
    eeg_acc, image_acc, avg_similarity, feature_class_acc, feature_id_acc = evaluate_system(phase='test')
else:
    eeg_acc, image_acc, avg_similarity, feature_class_acc, feature_id_acc = evaluate_out_system(phase='val')

EEG-Net val accuracy: 52.7778 %
Image-Net val accuracy: 98.2759 %
EEG-Image-Map avg val cosine similarity: 0.9217
EEG-Image-Map val classification accuracy: 44.636015 %
EEG-Image-Map val identification accuracy: 1.245211 %


In [ ]:
# Accuracy when selecting randomly
100/(len(feature_data_loaders['test'])*feature_hyperparams['batch_size'])

0.08223684210526316